<img align="center" src="https://raw.githubusercontent.com/FUlyankin/Parsers/master/images%20/cats.jpg" height="1200" width="1200"> 

# <center> ML для маркетинга. <br>  <br> Cюжетная арка 3: сегментация </center>

В этом семестре мы с вами пройдём через несколько сюжетных арок. Третьей и заключительной такой аркой будет сегментация. 


#  Глава II (Вконтакте и его мальчик) 

Ознакомиться со способами взаимодействия с Вконтакте можно по [ссылке](https://vk.com/dev/manuals) на отличную документацию. Для того, чтобы получить доступ к выгрузке данных, нужно пройти ряд бюрократических процедур (о, боже, эти два предложения были так бюрократически сформулированы, что мне захотелось отстоять в очереди).

Первая такая процедура заключается в создании своего приложения. Для этого переходим по [ссылке](http://vk.com/editapp?act=create) и проходимся по необходимым шагам:

<img align="center" src="pictures/app_creation_1.png" height="600" width="600">

После подтверждения своей личности по номеру телефона, попадаем на страницу свежесозданного приложения
<img align="center" src="pictures/app_creation_2.png" height="600" width="600">

Слева нам будем доступна вкладка с настройками, перейдя в неё мы увидим все необходимые нам для работы с приложением параметры:
<img align="center" src="pictures/app_creation_3.png" height="600" width="600">


Для работы с частью методов API этого вполне достаточно (обычно в заголовке такого метода стоит соответствующая пометка). Для части методов, используемых нами, может понадобиться ключ доступа.

Для того, чтобы получить его, необходимо сделать ещё пару странных манипуляций:

Переходим по ссылке вида (на месте звездочек должен стоять ID созданного вами приложения):

> https://oauth.vk.com/authorize?client_id=6666347&scope=14&redirect_uri=https://oauth.vk.com/blank.html&display=page&v=5.16&response_type=token

<img align="center" src="pictures/app_creation_4.png" height="600" width="600">

В итоге по этому запросу будет сформирована ссылка следующего вида:
> https://oauth.vk.com/blank.html#access_token=25b636116ef40e0718fe4d9f382544fc28&expires_in=86400&user_id=*******

Первый набор знаков - `access token`, т.е. маркер доступа. Вторая цифра (`expires_in=`) время работы маркера доступа в секундах (одни сутки). По истечению суток нужно будет получить новый маркер доступа. Последняя цифра (`user_id=`) ваш ID Вконтакте. Нам в дальнейшем понадобится маркер доступа. Для удобства сохраним его в отдельном файле или экспортируем в глобальную область видимости. В целях безопасности ваших данных не стоит нигде светить токенами и тем более выкладывать их в открытый доступ.

Обратите внимание на ссылку, по которой мы делали запрос на предоставление токена. Внутри неё находится странный параметр `scope=14.` Эта загадочная цифра есть ничто иное, как права доступа к социальной сети. Подробнее познакомиться с взаимно-однозначным соответствием между числами и правами можно в документации. Например, если мы хотим получить доступ к друзьям, фото и аудио, мы подставим в scope цифру 2+4+8=14.

> __ВАЖНО:__ Если отдать свой токен другому человеку, у него будет доступ к вашему аккаунту. Он легко может выкачать ваши личные переписки и даже удалить страницу. Не светите свой токен в интернетах и берегите его с молоду. 

# 1. Впервые врываемся в вк и узнаём как нас зовут

In [1]:
import requests # библиотека для походов в интернет

Подгрузим токен.

In [2]:
# мой номер странички
myid = '6045249'# вставить номер странички

# версия используемого API
version = '5.73' 

# подгружаем токен
with open('untitled.txt') as f:
    token = f.read()

In [3]:
token

'82b23bc582b23bc582e977325a82f8afb8882b282b23bc5da60d480cac91cc795a38d21'

Чтобы скачать что-то из контакта, надо сделать ссылку и сходить по ней пакетом `requests`. Ссылка должна будет включать в себя метод (что мы просим у вк) и параметры (насколько много). Мы будем просто заменять эти две штуки и выкачивать разные вещи. 

In [ ]:
method = 'users.get'
parameters = 'user_ids=' + myid

url = 'https://api.vk.com/method/' + method + '?' + parameters + '&access_token=' + str(token) + '&v=' + version

response = requests.get(url) 
response.json()

Если вы всё сделали правильно, то у вас выводится имя и фамилия.  Залезем в [документацию](http://vk.com/editapp?act=create) и скачаем что-нибудь по ней. Например, всех людей из группы: https://vk.com/hsemem

Если открыть первый пост в группе (нажать на дату его публикации), появиться ссылка https://vk.com/hsemem?w=wall-139105204_48261

Первые числа это и есть id.